In [ ]:

#  Check GPU availability
# Test if CUDA is available
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU device: Tesla T4


In [ ]:
# Create and setup hugging face API key in secrets (HF_TOKEN)

In [ ]:
# Install required dependencies
!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 834.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-o2rru3xk
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-o2rru3xk
  Resolved https://github.com/unslothai/unsloth.git to commit d8459b194c81a502962f16118509cdfa205483af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.12.9-py3-none-any.whl size=382476 sha256=5d741d88f235fd024f889de871fa4df87275dbe910cae3096192685ff20c24bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-svy9c7fu/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.12.9
    Uninstalling unsloth-2025.12.9:
      Successfully uninstalled unsloth-2025.12.9


In [ ]:
#  Import necessary libraries
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer # TRL supports the Supervised Fine-Tuning (SFT) Trainer for training language models.
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Check Hugging face token

# Check HF token
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
login(hf_token)


In [ ]:
# Check WANDB_API_TOKEN token
from google.colab import userdata

wandb_token = userdata.get('WANDB_API_TOKEN')

if wandb_token:
    print("WANDB_API_TOKEN successfully retrieved!")
    # For security reasons, I won't print the full token here,
    # but you can inspect the 'wandb_token' variable if needed.
    print(f"First 5 characters of WANDB_API_TOKEN: {wandb_token[:5]}...")
else:
    print("WANDB_API_TOKEN not found in Colab secrets. Please add it.")

WANDB_API_TOKEN successfully retrieved!
First 5 characters of WANDB_API_TOKEN: 
# S...


In [ ]:
# Setup pretrained deep seek r1
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
MAX_SEQUENCE_LENGTH = 2048
DTYPE = None # We keep it None , when we give data to our model it can check in which formatt he data it is getting reset
LOAD_IN_4BIT = True # Quantized model finetune


# Use model from  unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
    token =  hf_token
)

==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# To Test The Model (System Prompt)
prompt_style = """
Below is a task description along with additional context provided in the input section. Your goal is to provide an accurate, clinically sound response that directly addresses the request.

Use your medical expertise to ensure the answer is evidence-based, clearly structured, and easy to understand. When appropriate, briefly explain the rationale behind key conclusions without revealing internal reasoning processes.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.
Provide a clear, concise, and clinically appropriate response. Include relevant explanations, differential considerations, and recommendations as needed.

### Query:
{question}

### Answer:

"""


In [ ]:
# Ask a complicated question medical reasoning question from the model before finetuning

# Inference on the model
# question asked
question = """
A 61-year-old woman with a long history of involuntary urine leakage during activities such as coughing or sneezing,
but no nocturnal leakage, undergoes a gynecologic examination and a Q-tip test.
Based on these findings, what would cystometry most likely show regarding her postvoid residual volume and detrusor muscle contractions?

"""
# inference - Provide a clear, concise, and clinically appropriate response. Include relevant explanations, differential considerations, and recommendations as needed.
# Take inference with the help of unsloth
FastLanguageModel.for_inference(model)

# Tokenize the input
inputs = tokenizer([prompt_style.format(question=question)], return_tensors="pt").to("cuda")  # tokenize the input

# Generate a response
outputs = model.generate (
    input_ids = inputs.input_ids, # Think of this as the question you're giving to the AI model, but translated into numbers the computer understands.
    attention_mask = inputs.attention_mask, # Imagine you're highlighting the important parts of your question and crossing out any extra spaces or empty bits.(attention mask is used for padding)
    max_new_tokens = 1200, # Response Limit
    use_cache = True # if similar question is asked instead of running a whole , store it in cache fro there it will generate the output for fast response
)

# Decode the response tokens back to text
# response = tokenizer.batch_decode(outputs)

# Decode the response tokens back to text
# better response
# response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

In [ ]:
# Decode the response tokens back to text
response = tokenizer.batch_decode(outputs)
# Print full output
print(response)

["<｜begin▁of▁sentence｜>\nBelow is a task description along with additional context provided in the input section. Your goal is to provide an accurate, clinically sound response that directly addresses the request.\n\nUse your medical expertise to ensure the answer is evidence-based, clearly structured, and easy to understand. When appropriate, briefly explain the rationale behind key conclusions without revealing internal reasoning processes.\n\n### Task:\nYou are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.\nProvide a clear, concise, and clinically appropriate response. Include relevant explanations, differential considerations, and recommendations as needed.\n\n### Query:\n\nA 61-year-old woman with a long history of involuntary urine leakage during activities such as coughing or sneezing,\nbut no nocturnal leakage, undergoes a gynecologic examination and a Q-tip test.\nBased

In [ ]:
# Decode the response tokens back to text
# better response
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# Print full output
print(response)




Below is a task description along with additional context provided in the input section. Your goal is to provide an accurate, clinically sound response that directly addresses the request.

Use your medical expertise to ensure the answer is evidence-based, clearly structured, and easy to understand. When appropriate, briefly explain the rationale behind key conclusions without revealing internal reasoning processes.

### Task:
You are a medical expert specializing in clinical reasoning, diagnostics, and treatment planning. Answer the medical question below using your advanced knowledge.
Provide a clear, concise, and clinically appropriate response. Include relevant explanations, differential considerations, and recommendations as needed.

### Query:

A 61-year-old woman with a long history of involuntary urine leakage during activities such as coughing or sneezing,
but no nocturnal leakage, undergoes a gynecologic examination and a Q-tip test.
Based on these findings, what would cysto

In [ ]:
# Extract the answer part along with the think
answer = response.split("### Answer:")[-1].strip()
print(answer)

Cystometry, also known as urodynamic testing, is a crucial diagnostic tool for evaluating bladder function and urethral dynamics. In this case, the Q-tip test helps assess urethral competence, particularly in women, by determining if the urethral sphincter can properly close after urination.

The patient’s history of involuntary urine leakage during activities like coughing or sneezing suggests a potential issue with urethral sphincter function. The absence of nocturnal leakage is interesting because it implies that the leakage is not due to diurnal incontinence but rather related to activities that increase intra-abdominal pressure, such as coughing or sneezing.

During a Q-tip test, if the patient demonstrates urethral sphincter insufficiency, the Q-tip would remain elevated after voiding, indicating that the sphincter doesn't close properly. Conversely, if the sphincter is competent, the Q-tip would return to its normal position quickly after voiding.

Based on the findings from the

In [ ]:
#### Setup Fine tuning

In [ ]:
# Load the Data set
# medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]", trust_remote_code = True)
medical_dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[:500]")
# dataset - verified data used to train openai model (https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT)
# What the data set has (Question , chain of thought, response)
# "en" - English Language
# "train[:500]" :- It is a big data set so we will use the first 500 samples



README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [ ]:
# First Object
medical_dataset[0]


{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

In [ ]:
# when we are getting the output
# (response = tokenizer.batch_decode(outputs))
# so when we print the above it has <｜begin▁of▁sentence｜> () and <｜end▁of▁sentence｜> (It is used both for training and generating output)
# The tokens like <｜begin▁of▁sentence｜> and <｜end▁of▁sentence｜> are special tokens used by the model or tokenizer to mark the beginning and end of a generated sequence
# For finetuning, it's crucial to tell the model where the response ends with the help of this <｜end▁of▁sentence｜>

EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which tells the model when to stop generating text during training
EOS_TOKEN


'<｜end▁of▁sentence｜>'

In [ ]:
# Updated training prompt style to add </think> tag
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
# We have to format the data  in a specific format , so that we can give it to the model for training (Labelling)

# Prepare the data for finetuning
def preprocess_input_data(examples): # examples is typically a dictionary containing lists of values for each column
  inputs = examples["Question"] # This line extracts the list of questions from the examples dictionary and assigns it to the variable inputs
  cots= examples["Complex_CoT"] # similarly, this extracts the list of 'Complex_CoT' (Chain of Thought) from examples and assigns it to cots
  outputs = examples["Response"] # This extracts the list of desired responses and assigns it to responses

  texts = [] # An empty list called texts is initialized. This list will store the newly formatted strings that will be used for training.


  for input, cot, output in zip(inputs, cots, outputs): # The zip() function is like a zipper on a jacket. It takes these three lists and combines them item by item
    text = train_prompt_style.format(input, cot, output) + EOS_TOKEN # the input will automaticlly go in (### Questions {input}) in prompt
    texts.append(text)

  return {
      "texts" : texts,
  }



In [ ]:
# Convert the dataset to finetuning dataset
finetune_dataset = medical_dataset.map(preprocess_input_data , batched = True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Calling the first object of the text
finetune_dataset["texts"][0]
# the prompt has
# the question after the ### Question:{} ,
# Complex_CoT between <think>
# and Response after <think>


"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning.\nPlease answer the following medical question.\n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is sw

In [ ]:
# Save the fine-tuned model and tokenizer locally
model.save_pretrained("final_deepseek_model")
tokenizer.save_pretrained("final_deepseek_model")

In [ ]:
# Push the model and tokenizer to Hugging Face Hub
# Replace 'your-username/your-model-name' with your desired repository name
model.push_to_hub("your-username/deepseek-r1-distill-llama-8b-finetuned", token=hf_token)
tokenizer.push_to_hub("your-username/deepseek-r1-distill-llama-8b-finetuned", token=hf_token)